In [28]:
import  pandas as pd


dfs = pd.read_html("https://fbref.com/en/squads/206d90db/Barcelona-Stats")
print(dfs)


HTTPError: HTTP Error 429: Too Many Requests

# We look at the columns of each table

In [19]:
for i, df in enumerate(dfs):
    print(f"🔢 Tabla {i} - Shape: {df.shape}")
    print(f"🧩 Columnas: {list(df.columns)}\n")

🔢 Tabla 0 - Shape: (41, 34)
🧩 Columnas: [('Unnamed: 0_level_0', 'Player'), ('Unnamed: 1_level_0', 'Nation'), ('Unnamed: 2_level_0', 'Pos'), ('Unnamed: 3_level_0', 'Age'), ('Unnamed: 4_level_0', 'MP'), ('Playing Time', 'Starts'), ('Playing Time', 'Min'), ('Playing Time', '90s'), ('Performance', 'Gls'), ('Performance', 'Ast'), ('Performance', 'G+A'), ('Performance', 'G-PK'), ('Performance', 'PK'), ('Performance', 'PKatt'), ('Performance', 'CrdY'), ('Performance', 'CrdR'), ('Expected', 'xG'), ('Expected', 'npxG'), ('Expected', 'xAG'), ('Expected', 'npxG+xAG'), ('Progression', 'PrgC'), ('Progression', 'PrgP'), ('Progression', 'PrgR'), ('Per 90 Minutes', 'Gls'), ('Per 90 Minutes', 'Ast'), ('Per 90 Minutes', 'G+A'), ('Per 90 Minutes', 'G-PK'), ('Per 90 Minutes', 'G+A-PK'), ('Per 90 Minutes', 'xG'), ('Per 90 Minutes', 'xAG'), ('Per 90 Minutes', 'xG+xAG'), ('Per 90 Minutes', 'npxG'), ('Per 90 Minutes', 'npxG+xAG'), ('Unnamed: 33_level_0', 'Matches')]

🔢 Tabla 1 - Shape: (60, 20)
🧩 Columnas: ['

# Extract the tables that are useful to us

In [26]:
TABLAS_UTILES = [0, 2, 3, 4, 5, 7, 8, 9, 10, 11]
URL = "https://fbref.com/en/squads/53a2f082/2024-2025/Real-Madrid-Stats"

def extraer_tablas_utiles(URL: str, indices_utiles: list[int]) -> list[pd.DataFrame]:
    tablas = pd.read_html(URL, header=[0, 1])
    print(f"📊 Tablas totales encontradas: {len(tablas)}")
    
    tablas_utiles = []
    for i in indices_utiles:
        df = tablas[i].copy()
        print(f"\n✅ Tabla {i} seleccionada")
        print(f"Shape: {df.shape}")
        print(f"Columnas: {df.columns.tolist()[:5]} ...")
        tablas_utiles.append(df)
    
    return tablas_utiles

tablas = extraer_tablas_utiles(URL, TABLAS_UTILES)

for i, t in enumerate(tablas):
    print(f"✅ tabla[{i}] tipo: {type(t)}")


HTTPError: HTTP Error 429: Too Many Requests

# Merge into one dataframe with an outer product

In [27]:
from functools import reduce
import pandas as pd
import re

def flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    def limpiar_col(col):
        if isinstance(col, tuple):
            col = "_".join(col)
        # Elimina prefijos tipo "Unnamed: d_level_d_"
        col = re.sub(r"^Unnamed: \d+_level_\d+_", "", col) 
        return col.strip()
    
    df.columns = [limpiar_col(col) for col in df.columns]
    return df


def limpiar_tabla(df: pd.DataFrame) -> pd.DataFrame:
    df = flatten_columns(df)

    # Detectar columna que contiene 'Player'
    col_player = next((col for col in df.columns if 'Player' in col), None)
    if col_player is None:
        print("⚠️ No se encontró columna de jugador.")
        return None

    # Eliminar filas de cabecera o totales
    df = df[~df[col_player].isin(['Player'])]
    df = df.dropna(subset=[col_player])
    df = df.rename(columns={col_player: 'Player'})

    # Eliminar columna Matches si existe
    if 'Matches' in df.columns:
        df.drop(columns='Matches', inplace=True)

    return df


def merge_controlado_por_player(tablas: list[pd.DataFrame]) -> pd.DataFrame:
    tablas_limpias = [limpiar_tabla(df) for df in tablas if df is not None]
    df_base = tablas_limpias[0]

    for i, df_nueva in enumerate(tablas_limpias[1:], start=1):
        columnas_base = set(df_base.columns)
        columnas_nueva = set(df_nueva.columns)

        columnas_duplicadas = (columnas_base & columnas_nueva) - {'Player'}
        if columnas_duplicadas:
            print(f"🔁 Paso {i}: eliminando duplicadas → {columnas_duplicadas}")
            df_nueva = df_nueva.drop(columns=columnas_duplicadas, errors='ignore')

        columnas_nuevas = set(df_nueva.columns) - columnas_base
        print(f"➕ Paso {i}: nuevas columnas añadidas → {columnas_nuevas}")

        df_base = pd.merge(df_base, df_nueva, on='Player', how='outer')

    return df_base

df_final = merge_controlado_por_player(tablas)
print(f"✅ DataFrame final: {df_final.shape[0]} jugadores y {df_final.shape[1]} columnas")




🔁 Paso 1: eliminando duplicadas → {'Nation', 'Pos', 'Playing Time_Min', 'Age', 'Playing Time_MP', 'Playing Time_90s', 'Playing Time_Starts'}
➕ Paso 1: nuevas columnas añadidas → {'Performance_W', 'Performance_GA', 'Penalty Kicks_PKatt', 'Performance_D', 'Penalty Kicks_PKsv', 'Penalty Kicks_Save%', 'Performance_GA90', 'Performance_CS', 'Performance_CS%', 'Penalty Kicks_PKm', 'Performance_Saves', 'Performance_SoTA', 'Penalty Kicks_PKA', 'Performance_Save%', 'Performance_L'}
🔁 Paso 2: eliminando duplicadas → {'Nation', 'Age', 'Pos'}
➕ Paso 2: nuevas columnas añadidas → {'Launched_Cmp%', 'Expected_/90', 'Goals_GA', 'Passes_Thr', 'Sweeper_AvgDist', 'Expected_PSxG+/-', 'Launched_Att', 'Sweeper_#OPA/90', 'Crosses_Stp', 'Goal Kicks_Launch%', 'Passes_Att (GK)', 'Crosses_Stp%', 'Goal Kicks_AvgLen', 'Sweeper_#OPA', 'Goals_OG', 'Goals_FK', 'Passes_Launch%', 'Expected_PSxG', 'Goals_PKA', 'Crosses_Opp', '90s', 'Launched_Cmp', 'Goal Kicks_Att', 'Passes_AvgLen', 'Goals_CK', 'Expected_PSxG/SoT'}
🔁 Paso

# Column cleaning

Index(['Player', 'Unnamed: 1_level_0_Nation', 'Unnamed: 2_level_0_Pos',
       'Unnamed: 3_level_0_Age', 'Playing Time_MP', 'Playing Time_Starts',
       'Playing Time_Min', 'Playing Time_90s', 'Performance_Gls',
       'Performance_Ast',
       ...
       'Performance_Crs', 'Performance_Int', 'Performance_TklW',
       'Performance_PKwon', 'Performance_PKcon', 'Performance_OG',
       'Performance_Recov', 'Aerial Duels_Won', 'Aerial Duels_Lost',
       'Aerial Duels_Won%'],
      dtype='object', length=207)

# Extrapolar a todos los equipos 

In [24]:
import time
import pandas as pd
from pathlib import Path

# Carpeta donde se guardarán los CSV
EXPORT_PATH = Path("data/processed/")
EXPORT_PATH.parent.mkdir(parents=True, exist_ok=True)

la_liga = pd.read_html("https://fbref.com/en/comps/12/La-Liga-Stats", extract_links='all')[0]

equipos_urls = {}
for fila in df[(('Squad', None))].dropna():
    nombre_equipo, enlace = fila
    if enlace:
        equipos_urls[nombre_equipo.strip()] = f"https://fbref.com{enlace}"

print(f"✅ Se encontraron {len(equipos_urls)} equipos")

for nombre, url in equipos_urls.items():
    print(f"\n📥 Procesando {nombre} → {url}")
    
    # 1. Extraer tablas útiles
    tablas_equipo = extraer_tablas_utiles(url, TABLAS_UTILES)
    
    # 2. Fusionar por jugador
    df_equipo = merge_controlado_por_player(tablas_equipo)
    
    # 3. Guardar CSV con nombre del equipo
    nombre_archivo = f"{nombre.replace(' ', '_')}.csv"
    ruta_archivo = EXPORT_PATH / nombre_archivo
    df_equipo.to_csv(ruta_archivo, index=False)
    
    print(f"✅ {nombre} guardado en {ruta_archivo}")
    
    # 4. Esperar para evitar bloqueo
    time.sleep(5)





HTTPError: HTTP Error 429: Too Many Requests

In [ ]:

import pathlib
from bs4 import BeautifulSoup
import  pandas as pd
import re
import requests
from datetime import datetime


HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/125.0.0.0 Safari/537.36"
    )
}

URL_CLUBS = "https://www.transfermarkt.com/laliga/daten/wettbewerb/ES1"
OUT_PATH = pathlib.Path("data/raw/transfermarkt_laliga_clubs.html")

def fetch_market_value_history_api(player_url: str) -> pd.DataFrame:
    # Extraer player_id desde la URL del jugador
    m = re.search(r"/spieler/(\d+)", player_url)
    if not m:
        print("❌ No se pudo extraer player_id de la URL:", player_url)
        return pd.DataFrame()
    player_id = m.group(1)

    api_url = f"https://www.transfermarkt.com/ceapi/marketValueDevelopment/graph/{player_id}"
    print(f"🔗 Accediendo a la API para {player_id} en {api_url}")

    try:
        resp = requests.get(api_url, headers=HEADERS, timeout=20)
        resp.raise_for_status()
    except Exception as e:
        print(f"❌ Error al acceder a la API para {player_id}: {e}")
        return pd.DataFrame()

    json_data = resp.json()

    if "list" not in json_data:
        print("⚠️ No se encontró la clave 'list' en la respuesta de la API")
        return pd.DataFrame()

    rows = []
    for point in json_data["list"]:
        timestamp = point.get("x")  # milisegundos
        value = point.get("y")
        date = datetime.utcfromtimestamp(timestamp // 1000).date()
        club = point.get("verein", "")
        age = point.get("age", "")
        mv_str = point.get("mw", "")

        rows.append({
            "date": date,
            "market_value_million": value / 1e6,
            "market_value_str": mv_str,
            "club": club,
            "age": age
        })

    df = pd.DataFrame(rows)
    return df



# Asumiendo que ya tienes player_url y nombre
player_url = "https://www.transfermarkt.com/vinicius-junior/profil/spieler/371998"
df_mv = fetch_market_value_history_api(player_url)
print(df_mv.head())

# Guardar como CSV
player_name = "vinicius_junior"
out_path = pathlib.Path(f"data/processed/market_values/{player_name}.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)
df_mv.to_csv(out_path, index=False)


🔗 Accediendo a la API para 371998 en https://www.transfermarkt.com/ceapi/marketValueDevelopment/graph/371998
         date  market_value_million market_value_str         club age
0  2017-10-29                  10.0          €10.00m  CR Flamengo  17
1  2017-12-06                  20.0          €20.00m  CR Flamengo  17
2  2018-05-22                  35.0          €35.00m  CR Flamengo  17
3  2018-12-20                  35.0          €35.00m  Real Madrid  18
4  2019-02-27                  70.0          €70.00m  Real Madrid  18


C:\Users\jorgi\AppData\Local\Temp\ipykernel_3892\309705292.py:48: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.utcfromtimestamp(timestamp // 1000).date()


: 